## Optimizing `significance_level` parameter of `KerasSiamesePageDetector`

In [1]:
from itertools import product

from video699.configuration import get_configuration
from video699.video.annotated import (
    AnnotatedSampledVideoScreenDetector,
    evaluate_event_detector,
    get_videos,
)
from video699.page.siamese import KerasSiamesePageDetector
from video699.quadrangle.rtree import RTreeDequeConvexQuadrangleTracker
from video699.event.screen import ScreenEventDetector

import numpy as np


CONFIGURATION = get_configuration()['KerasSiamesePageDetector']

Using TensorFlow backend.


In [2]:
def accuracy(significance_level):
    CONFIGURATION['significance_level'] = str(significance_level)
    num_successes_total = 0
    num_trials_total = 0
    for annotated_video in get_videos().values():
        convex_quadrangle_tracker = RTreeDequeConvexQuadrangleTracker(2)
        screen_detector = AnnotatedSampledVideoScreenDetector()
        documents = annotated_video.documents.values()
        page_detector = KerasSiamesePageDetector(documents)
        screen_event_detector = ScreenEventDetector(
            annotated_video,
            convex_quadrangle_tracker,
            screen_detector,
            page_detector
        )
        num_successes, num_trials = evaluate_event_detector(annotated_video, screen_event_detector)
        num_successes_total += num_successes
        num_trials_total += num_trials
    accuracy = 1.0 * num_successes_total / num_trials_total
    return accuracy

In [3]:
%%time
significance_levels = np.linspace(0.0, 1.0, num=50)
accuracies = list(map(accuracy, significance_levels))

best_accuracy, best_significance_level = max(zip(accuracies, significance_levels))
print('Optimal parameters (accuracy {}):'.format(best_accuracy))
print('- significance_level: {}'.format(best_significance_level))

Optimal parameters (accuracy 0.8557457212713936):
- significance_level: 0.42857142857142855
CPU times: user 3h 27min 16s, sys: 1h 2min 40s, total: 4h 29min 56s
Wall time: 3h 48min 48s
